<h1><u>Entire submission has been done in this single notebook

<h6>Import essential libraries

In [43]:
#wikipedia module doesn't work...

#import wikipedia

import pandas as pd
import numpy as np

In [44]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

<h3>Pull data from wiki</h3>

<ul>
    <li>populate a dataframe with the data from wiki</li>
    <li>exclude all results where Borough is not assigned</li>
    <li>populate Neighbourhood with Borough if not assigned</li>

In [45]:
df = pd.read_html(url, header = 0)[0]
df2 = df[df['Borough'] != 'Not assigned'].copy()
df2['Neighbourhood'] = np.where(df2['Neighbourhood'] == 'Not assigned', df2['Borough'], df2['Neighbourhood'])
df2.index = range(len(df2))

<ul>
    <li>Create an empty dataframe we're going to populate with transformed data</li>

In [46]:
df3 = pd.DataFrame(columns = ['PostalCode', 'Borough', 'Neighbourhood'])

<h3>Collapse columns to rows</h3>

<ul>
    <li>Loop through unique postcodes in dataset</li>
    <li>return the series of Neighbourhood & concatenate these separated by ', '</li>
    <li>populate our dataframe with a listc onatining postcode, borough & neighbourhoods</li>

In [47]:
for pc in df2['Postcode'].unique():
    
    s = df2[df2['Postcode'] == pc]['Neighbourhood']
    z = s.str.cat(sep = ', ')
    b = df2[df2['Postcode'] == pc]['Borough'].iloc[0]
    
    df3.loc[len(df3), :] = [pc, b, z]

<ul>
    <li>Select example post codes</li>
    <li>No sort order specified so stick with default

In [48]:
example_pc = ['M5G', 'M2H', 'M4B', 'M1J', 'M4G', 'M4M', 'M1R', 'M9V', 'M9L', 'M5V', 'M1B', 'M5A']

In [49]:
df3[df3['PostalCode'].isin(example_pc)]

,PostalCode,Borough,Neighbourhood
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
6,M1B,Scarborough,"Rouge, Malvern"
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
23,M4G,East York,Leaside
24,M5G,Downtown Toronto,Central Bay Street
27,M2H,North York,Hillcrest Village
32,M1J,Scarborough,Scarborough Village
50,M9L,North York,Humber Summit
54,M4M,East Toronto,Studio District
71,M1R,Scarborough,"Maryvale, Wexford"


<ul>
    <li>Show dimensions of dataframe

In [50]:
df3.shape

(103, 3)

<h3>Getting latitude & longitude for postcodes</h3>

<li>Geocoder call really slow though so just using provided csv...

In [51]:
geo_data = 'http://cocl.us/Geospatial_data'

geo = pd.read_csv(geo_data)
geo.columns, len(geo)

(Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object'), 103)

In [52]:
df4 = df3.merge(geo,left_on = 'PostalCode', right_on = 'Postal Code', how = 'inner')\
    .drop('Postal Code', axis = 1)

len(df3), len(df4)

(103, 103)

In [53]:
df4[df4['PostalCode'].isin(example_pc)]

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
23,M4G,East York,Leaside,43.709060,-79.363452
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
27,M2H,North York,Hillcrest Village,43.803762,-79.363452
32,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
50,M9L,North York,Humber Summit,43.756303,-79.565963
54,M4M,East Toronto,Studio District,43.659526,-79.340923
71,M1R,Scarborough,"Maryvale, Wexford",43.750072,-79.295849


<h3>Clustering neighbourhoods</h3>

<ul>
    <li>Replicating NY analysis in Toronto</li>
    <li>For improved granularity working with initial dataset & getting geo co-ordinates for each neighborhood rather than borough</li>

In [54]:
%matplotlib inline

from geopy.geocoders import Nominatim

import matplotlib.pyplot as plt
from matplotlib import cm, colors

plt.rcParams['figure.figsize'] = 17, 8

import folium
import requests

from sklearn.cluster import KMeans

<li>Getting all Toronto neighborhoods 

In [55]:
toronto_neighbourhoods = df2[df2['Borough'].str.lower().str.contains('toronto')].copy()
toronto_neighbourhoods.index = range(len(toronto_neighbourhoods))

In [56]:
toronto_neighbourhoods['address'] = toronto_neighbourhoods['Neighbourhood'] + ', Toronto'

In [57]:
toronto_geo = pd.DataFrame(columns = ['address', 'latitude', 'longitude'])

In [58]:
geolocator = Nominatim(user_agent="_explorer")

<li>Get lat & long for neighborhoods

In [59]:
for addr in toronto_neighbourhoods['address'].unique():    
    try:
        location = geolocator.geocode(addr)
        latitude = location.latitude
        longitude = location.longitude
        
        toronto_geo.loc[len(toronto_geo), :] = [addr, latitude, longitude]
    except:
        #Just skip records that don't return a location
        pass

In [60]:
toronto = toronto_neighbourhoods.merge(toronto_geo, on = 'address', how = 'inner').drop_duplicates()

<li>Visulise these points using folium

In [61]:
location = geolocator.geocode('Toronto')
latitude = location.latitude
longitude = location.longitude

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(toronto['latitude'], toronto['longitude'], 
                                           toronto['Borough'], toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [62]:
CLIENT_ID = '' # your Foursquare I
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

<li>Pull data from foursquare

In [63]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT = 100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):        
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [64]:
toronto_venues = getNearbyVenues(names=toronto['Neighbourhood'],
                                   latitudes=toronto['latitude'],
                                   longitudes=toronto['longitude']
                                  )

<li>Just see how many values where pulled for each Neighbourhood

In [65]:
toronto_venues.groupby(['Neighborhood']).size().reset_index(name = 'cs').sort_values('cs', ascending = False).head()

,Neighborhood,cs
0,Adelaide,100
14,Design Exchange,100
65,Trinity,100
63,Toronto Dominion Centre,100
33,King and Spadina,100


<li>One hot encode each venue category, this allows these fields to be passed to the kmeans algo

In [66]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

<ul>
<li>Leave as five clusters as in exercise</li>
<li>For a proper project would look to identify optimal N

In [77]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 1, 0, 2, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 4, 4, 4,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 3, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 4, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 1])

<li>Just check the cluster centres

In [78]:
kmeans.cluster_centers_

array([[0.00000000e+00, 4.76190476e-04, 1.08420217e-19, ...,
        4.33680869e-19, 4.76190476e-04, 1.70068027e-03],
       [1.74825175e-03, 1.08022186e-03, 4.91543340e-04, ...,
        1.97375678e-03, 3.61268939e-03, 3.17062205e-03],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [79]:
toronto_grouped.insert(0, 'Clust', kmeans.labels_)

In [80]:
clusts = toronto.merge(toronto_grouped[['Neighborhood', 'Clust']],
             left_on = 'Neighbourhood',
             right_on = 'Neighborhood',
             how = 'inner')\
    .drop(['address', 'Neighborhood'], axis = 1)

<ul>
    <li>Would suggest that five is not an optimal number of clusters & a further investigation would be required

In [81]:
clusts.groupby(['Clust'])['Neighbourhood'].nunique()

Clust
0    21
1    44
2     1
3     1
4     4
Name: Neighbourhood, dtype: int64

<li>Visualise all clusters using Folium

In [82]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(clusts['latitude'], clusts['longitude'], 
                                  clusts['Neighbourhood'], clusts['Clust']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters